In [11]:
from pathlib import Path


In [22]:
path = Path('/data/home/webb/file.csv')
path

PosixPath('/data/home/webb/file.csv')

In [23]:
path.with_suffix('.pkl')

PosixPath('/data/home/webb/file.pkl')

In [24]:
path.with_suffix('.csv')

PosixPath('/data/home/webb/file.csv')

In [28]:
# python
import pandas as pd

s = pd.Series([
    pd.Timedelta(minutes=5),
    pd.Timedelta(minutes=7),
    pd.Timedelta(minutes=6),
    pd.NaT  # example missing value
])
s

0   0 days 00:05:00
1   0 days 00:07:00
2   0 days 00:06:00
3               NaT
dtype: timedelta64[ns]

In [27]:

# Timedelta mean (skipna=True by default)
mean_td = s.mean()
print(mean_td)  # e.g. 0 days 00:06:00

# Numeric mean in seconds (useful for further calculations)
mean_seconds = s.dt.total_seconds().mean()
print(mean_seconds)  # e.g. 360.0

# Convert numeric mean back to Timedelta
mean_td_from_seconds = pd.to_timedelta(mean_seconds, unit='s')
print(mean_td_from_seconds)

0 days 00:06:00
360.0
0 days 00:06:00


In [10]:
from pandas import Timedelta, Series, DataFrame

ds = [
    Timedelta('06:21:02.234'),
    Timedelta('04:00:01.234345'),
]
ds = Series(ds)
ds.apply(lambda x: str(x.to_pytimedelta()))

0    6:21:02.234000
1    4:00:01.234345
dtype: object

In [25]:
import time
from data_cleaning.list_rename_move_edf_data import read_edf_time_info
from config.paths import PATHS, PatientDir

pd = PatientDir(PATHS.for_mayo_dir / 'B52K3P3G')
edf_path = list(pd.edf_dir.iterdir())[0]

st = time.perf_counter()
for _ in range(10000):
    read_edf_time_info(edf_path, is_competition_ptnt=False)
et = time.perf_counter() - st
print(et)

21.317613011226058


In [28]:
from data_cleaning.list_rename_move_edf_data import timezone_from_edf_header
import pyedflib

header = pyedflib.highlevel.read_edf_header(str(edf_path))
tz_location = timezone_from_edf_header(header)
tz_location

'UTC+02:00'